In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Dataset.csv')
df.head()

,Unnamed: 0,over,ball,total_runs,wicket left,runs required,winner,season
0,0,1,1,1,10.0,206.0,0.0,2017
1,1,1,2,1,10.0,206.0,0.0,2017
2,2,1,3,1,10.0,206.0,0.0,2017
3,3,1,4,3,10.0,204.0,0.0,2017
4,4,1,5,7,10.0,200.0,0.0,2017


In [3]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
df.head()

,over,ball,total_runs,wicket left,runs required,winner,season
0,1,1,1,10.0,206.0,0.0,2017
1,1,2,1,10.0,206.0,0.0,2017
2,1,3,1,10.0,206.0,0.0,2017
3,1,4,3,10.0,204.0,0.0,2017
4,1,5,7,10.0,200.0,0.0,2017


In [5]:
def create_dataset(dataset,time_stamp=6):
    X,Y = [],[]
    for i in range(len(dataset)-time_stamp-1):
        a = dataset[i:(i+time_stamp),:-1]
        X.append(a)
        Y.append(dataset[i+time_stamp,-1])
    return np.array(X), np.array(Y)

In [6]:
train_df = df[df['season']<=2017]
test_df = df[df['season']>2017]

In [7]:
train_df.drop(['season'],axis=1,inplace=True)
test_df.drop(['season'],axis=1,inplace=True)

In [8]:
stamp=1
x_train, y_train = create_dataset(train_df.values, time_stamp=stamp)

x_test, y_test = create_dataset(test_df.values, time_stamp=stamp)

In [9]:
x_train.shape

(70487, 1, 5)

In [17]:
model = Sequential()
model.add(LSTM(2, return_sequences=True, input_shape=(1, 5)))
model.add(LSTM(2, return_sequences=True))
model.add(LSTM(2))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=20,validation_data=(x_test, y_test), batch_size=120)

Train on 70487 samples, validate on 13442 samples
Epoch 1/20
70487/70487 [==============================] - 13s 182us/sample - loss: 0.6874 - acc: 0.5410 - val_loss: 0.6797 - val_acc: 0.5472
Epoch 2/20
70487/70487 [==============================] - 3s 46us/sample - loss: 0.6744 - acc: 0.5627 - val_loss: 0.6686 - val_acc: 0.5234
Epoch 3/20
70487/70487 [==============================] - 3s 46us/sample - loss: 0.5663 - acc: 0.7074 - val_loss: 0.6081 - val_acc: 0.6908
Epoch 4/20
70487/70487 [==============================] - 3s 47us/sample - loss: 0.5109 - acc: 0.7724 - val_loss: 0.6034 - val_acc: 0.6936
Epoch 5/20
70487/70487 [==============================] - 3s 48us/sample - loss: 0.5057 - acc: 0.7738 - val_loss: 0.6188 - val_acc: 0.6819
Epoch 6/20
70487/70487 [==============================] - 3s 48us/sample - loss: 0.5045 - acc: 0.7736 - val_loss: 0.5936 - val_acc: 0.7070
Epoch 7/20
70487/70487 [==============================] - 3s 48us/sample - loss: 0.5013 - acc: 0.7753 - val_loss: 

In [18]:
model.save('ipl.h5')